In [29]:
import tensorflow.compat.v1 as tf
import keras
from keras import applications
from keras.utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import argparse

In [7]:
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-o", "--output", required=True,
    help="path to output plot")
ap.add_argument("-g", "--gpus", type=int, default=1,
    help="# of GPUs to use for training")
args = vars(ap.parse_args())
# grab the number of GPUs and store it in a conveience variable
G = args["gpus"]

usage: ipykernel_launcher.py [-h] -o OUTPUT [-g GPUS]
ipykernel_launcher.py: error: the following arguments are required: -o/--output


SystemExit: 2

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [18]:
args = {"gpus":0, "output": "out_test.png"}
G = args["gpus"]

In [24]:
labels = np.load('labels.npy')
animals = np.load('label_to_animal.npz')
animals = animals['arr_0']
for i in animals:
    print(i)
NUM_CLASSES = animals.shape[0]

bobcat
opossum
empty
coyote
raccoon
bird
dog
cat
squirrel
rabbit
skunk
rodent
badger
deer
car
fox


In [23]:
images = np.load('images.npy')
print(images.shape)
INPUT_SHAPE = images[0].shape

(1000, 224, 224, 3)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1)

In [30]:
# definine the total number of epochs to train for along with the
# initial learning rate
def poly_decay(epoch):
    # initialize the maximum number of epochs, base learning rate,
    # and power of the polynomial
    maxEpochs = NUM_EPOCHS
    baseLR = INIT_LR
    power = 1.0
    # compute the new learning rate based on polynomial decay
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
    # return the new learning rate
    return alpha

# construct the image generator for data augmentation and construct
# the set of callbacks
aug = ImageDataGenerator(width_shift_range=0.1,
    height_shift_range=0.1, horizontal_flip=True,
    fill_mode="nearest")
callbacks = [LearningRateScheduler(poly_decay)]

In [60]:
y_lst = list(np.argmax(y_train, axis=1))
unique, counts = np.unique(np.argmax(y_train, axis=1), return_counts=True)
class_weights = {unique[i]:counts[i] for i in range(len(unique))}
for i in range(NUM_CLASSES):
    if i not in class_weights:
        class_weights[i] = 0

In [12]:
base_model = applications.resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=INPUT_SHAPE)
model.save("base_model.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/opt/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [16]:
DROPOUT = 0.1
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(DROPOUT)(x)
predictions = keras.layers.Dense(NUM_CLASSES, activation= 'softmax')(x)
def model():
    return keras.models.Model(inputs = base_model.input, outputs = predictions)

NameError: name 'NUM_CLASSES' is not defined

In [31]:
from keras.optimizers import Adam

# check to see if we are compiling using just a single GPU
if G <= 1:
    print("[INFO] training with 1 GPU...")
    model = model()
else:
    # otherwise, we are compiling using multiple GPUs
    print("[INFO] training with {} GPUs...".format(G))
    # we'll store a copy of the model on *every* GPU and then combine
    # the results from the gradient updates on the CPU
    with tf.device("/cpu:0"):
        # initialize the model
        model = model()
    
    # make the model parallel
    model = multi_gpu_model(model, gpus=G)

In [ ]:
print("[INFO] compiling model...")
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
NUM_EPOCHS = 10
INIT_LR = 5e-3
BATCH_SIZE = 32

# train the network
print("[INFO] training network...")
H = model.fit_generator(
    aug.flow(X_train, y_train, batch_size=BATCH_SIZE * G),
    validation_data=(X_test, y_test),
    class_weight=class_weights
    steps_per_epoch=len(X_train) // (BATCH_SIZE * G),
    epochs=NUM_EPOCHS,
    callbacks=callbacks, verbose=2)

"""
model.fit(X_train, y_train, epochs = 10, batch_size = 32,
          validation_data=(X_test, y_test))
"""

Train on 700 samples, validate on 300 samples
Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[32,125,250,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node res2a_branch1_1/BiasAdd-0-6-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_180876]

Function call stack:
keras_scratch_graph


In [ ]:
print('\n# Evaluate on test data')
results = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print('test loss, test acc:', results)

In [ ]:
model.save("model.h5")